# wide_dnn_eager_test3_store_json

* xor 해결 방법을 이용해 dnn 학습

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_row', 40000)
pd.set_option('display.max_column', 10000)
import matplotlib.pyplot as plt
%matplotlib inline
from os.path import join
import tensorflow as tf
# tf 2.0부터 keras는 tensorflow의 공식 API
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical
import sqlite3
import glob
import os
import csv

## Load Data

In [2]:
path = "src/*" #수정
file_list =  glob.glob(path)
file_list_db = [file for file in file_list if file.endswith(".db")]
print ("file_list: {}".format(file_list_db))
len(file_list_db)
# EventCode03이 일어났을 당시의 초당 주행기록 가져오기
srcrec_df2 = pd.DataFrame() #연결한 db 결과 저장소
for i in range (0,len(file_list_db)):
    # event 파일 가져오기
    f = open('src/event.csv') #수정
    csvReader = csv.reader(f)
    #db연결
    conn = sqlite3.connect(file_list_db[i])
    c = conn. cursor()
    # event 테이블 유무 확인 후, 있으면 제거
    c.execute('Drop Table If Exists event')
    # event 테이블 생성
    c.execute("create table event(CAR_RECDRV_KEY integer, EVENT_CODE text, EVENT_STDT text, EVENT_ENDT text)")
    # csv 파일 읽어 온 데이터 insert
    for row in csvReader:
        if row[7] == "EVENT_CODE":
            continue
        sql1 = "insert into event (CAR_RECDRV_KEY,EVENT_CODE, EVENT_STDT, EVENT_ENDT) values (?,?,?,?)"
        key= int(row[1])
        #key 추출
        if key != int(file_list_db[i][4:11]):
            continue
        code = (row[7])
        #event 추출
        if code[-11:] != "EventCode03":
            continue
        stdt = (row[8])
        endt = (row[9])
        c.execute(sql1,(key,code,stdt,endt))
    #트랜잭션 저장
    conn.commit()
    #event03 뽐기 query 실행
    sql2 = 'SELECT SRCREC.srcValue, SRCREC.realTime, SRCREC.srcSpeed, SRCREC.srcAPS,\
    SRCREC.srcGyroValue, SRCREC.srcRPM, SRCREC.srcTPS, SRCREC.srcMAF, SRCREC.srcEngineLoad, ifnull(EVENT_CODE, "0") FROM SRCREC LEFT OUTER JOIN\
    (SELECT EVENT_CODE, EVENT_STDT s, EVENT_ENDT e FROM event\
    ) ON SRCREC.realTime BETWEEN strftime("%Y%m%d%H%M%S",s,"-6 seconds")\
     AND strftime("%Y%m%d%H%M%S",s,"-1 seconds")'
    query =  c.execute(sql2)
    cols = [column[0] for column in query.description]
    srcrec_df = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)
    #db 파일 연결 작업
    srcrec_df2 = pd.concat([srcrec_df2, srcrec_df], ignore_index=True)
    #db 연결 종료
    c.close()
    conn.close()
    #파일 연결 종료
    f.close()
srcrec_df = srcrec_df2

file_list: ['src\\3048682.db']


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

tf.random.set_seed(777)  # for reproducibility

In [4]:
x_data = srcrec_df.apply(pd.to_numeric,errors='coerce')
x_data = x_data.iloc[:,2:9]
x_data = x_data.values
x_data

array([[ 4.        , 14.90196   ,  5.64022684, ..., 14.117647  ,
         0.        ,  0.        ],
       [ 3.        , 14.90196   ,  6.97546625, ..., 14.117647  ,
         0.        , 49.80392   ],
       [ 3.        , 14.90196   ,  5.53535843, ..., 14.117647  ,
         0.        , 49.80392   ],
       ...,
       [ 0.        , 14.90196   , -0.08611873, ..., 12.156863  ,
         0.        , 38.039215  ],
       [ 0.        , 14.90196   ,  0.48863536, ..., 12.156863  ,
         0.        , 38.039215  ],
       [ 0.        , 14.90196   ,  0.28494966, ..., 12.156863  ,
         0.        , 39.215687  ]])

In [5]:
y_data = srcrec_df.values[:,9]
y_data = pd.get_dummies(y_data).values

y_data = y_data[:,0]

where_0 = np.where(y_data ==0)
where_1 = np.where(y_data ==1)

y_data[where_0] = 1
y_data[where_1] = 0

y_data = y_data.reshape(-1,1)
y_data

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=uint8)

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))
dataset

<BatchDataset shapes: ((None, 7), (None, 1)), types: (tf.float64, tf.uint8)>

In [7]:
nb_classes = 10
mylist = []

class wide_deep_nn():
    def __init__(self, nb_classes):
        #딥러닝 적용 부분
        super(wide_deep_nn, self).__init__()        
         #숫자 바꾸기
        self.W1 = tf.Variable(tf.random.normal((7, nb_classes)), name='weight1')
        self.b1 = tf.Variable(tf.random.normal((nb_classes,)), name='bias1')

        self.W2 = tf.Variable(tf.random.normal((nb_classes, nb_classes)), name='weight2')
        self.b2 = tf.Variable(tf.random.normal((nb_classes,)), name='bias2')

        self.W3 = tf.Variable(tf.random.normal((nb_classes, nb_classes)), name='weight3')
        self.b3 = tf.Variable(tf.random.normal((nb_classes,)), name='bias3')

        self.W4 = tf.Variable(tf.random.normal((nb_classes, 1)), name='weight4')
        self.b4 = tf.Variable(tf.random.normal((1,)), name='bias4')
        
        self.variables = [self.W1,self.b1,self.W2,self.b2,self.W3,self.b3,self.W4,self.b4]
        
    def preprocess_data(self, features, labels):
        features = tf.cast(features, tf.float32)
        labels = tf.cast(labels, tf.float32)
        return features, labels
        
    def deep_nn(self, features):
        layer1 = tf.sigmoid(tf.matmul(features, self.W1) + self.b1)
        layer2 = tf.sigmoid(tf.matmul(layer1, self.W2) + self.b2)
        layer3 = tf.sigmoid(tf.matmul(layer2, self.W3) + self.b3)
        hypothesis = tf.sigmoid(tf.matmul(layer3, self.W4) + self.b4)
        return hypothesis
    
    def loss_fn(self, hypothesis, features, labels):
        cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
        return cost

    def accuracy_fn(self, hypothesis, labels):
        predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
        for i in range (0,len(predicted)):
            mylist.append(predicted[i])
        print("predicted : ",predicted,"\nlabels : ",labels,"\n equal : ",tf.equal(predicted,labels))
        accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
        return accuracy

    def grad(self, hypothesis, features, labels):
        with tf.GradientTape() as tape:
            loss_value = self.loss_fn(self.deep_nn(features),features,labels)
        return tape.gradient(loss_value,self.variables)
    
    def fit(self, dataset, EPOCHS=1000, verbose=100):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.01)
        for step in range(EPOCHS):
            for features, labels  in dataset:
                features, labels = self.preprocess_data(features, labels)
                grads = self.grad(self.deep_nn(features), features, labels)
                optimizer.apply_gradients(grads_and_vars=zip(grads, self.variables))
                if step % verbose == 0:
                    print("Iter: {}, Loss: {:.4f}".format(step, self.loss_fn(self.deep_nn(features),features,labels)))

    def test_model(self,x_data, y_data):
        x_data, y_data = self.preprocess_data(x_data, y_data)
        test_acc = self.accuracy_fn(self.deep_nn(x_data),y_data)
        
        #print("Testset Accuracy: {:.4f}".format(test_acc))


In [8]:
model = wide_deep_nn(nb_classes)

In [9]:
model.fit(dataset)

Iter: 0, Loss: 1.5481
Iter: 100, Loss: 0.2941
Iter: 200, Loss: 0.2143
Iter: 300, Loss: 0.1941
Iter: 400, Loss: 0.1865
Iter: 500, Loss: 0.1831
Iter: 600, Loss: 0.1813
Iter: 700, Loss: 0.1804
Iter: 800, Loss: 0.1799
Iter: 900, Loss: 0.1796


In [10]:
model.test_model(x_data, y_data)

predicted :  tf.Tensor(
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]], shape=(2341, 1), dtype=float32) 
labels :  tf.Tensor(
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]], shape=(2341, 1), dtype=float32) 
 equal :  tf.Tensor(
[[ True]
 [ True]
 [ True]
 ...
 [ True]
 [ True]
 [ True]], shape=(2341, 1), dtype=bool)


In [11]:
mylist = np.array(mylist)
mylist = list([int(x) for x in mylist])
mylist

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


### loss가 0.02까지 낮아짐. accuracy는 0.9979

In [12]:
import json

#json파일 저장경로 설정
file_path ="./result.json" 

data = {}
data['status'] = []

for i in range(0, len(mylist)):
    data['status'].append(mylist[i])

#json형식으로 데이터 저장
with open(file_path, 'w') as outfile:
    json.dump(data, outfile)

# 저장한 파일 출력하기

with open(file_path, 'r') as f:
    json_data = json.load(f)

print(json.dumps(json_data, indent="\t") )

{
	"status": [
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
		0,
